<a href="https://colab.research.google.com/github/thedataninja1786/Data-Science/blob/main/LRFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TEXT PREPROCESSING AND CLASSIFICATION FROM SCRATCH

In [ ]:
#Import necessary modules 
import nltk
import re 
import string 
import pandas as pd 
import numpy as np 
nltk.download('twitter_samples')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
positive_tweets =twitter_samples.strings('positive_tweets.json')
negative_tweets =twitter_samples.strings('negative_tweets.json')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Load the train and test data 
positive_tweets =twitter_samples.strings('positive_tweets.json')
negative_tweets =twitter_samples.strings('negative_tweets.json')

positive_train_tweets = positive_tweets[:4000]
positive_test_tweets = positive_tweets[4000:]
negative_train_tweets = negative_tweets[:4000]
negative_test_tweets = negative_tweets[4000:]

#Assign label 1 for positive tweets and 0 for negative tweets
positive_df = pd.DataFrame(positive_train_tweets) 
positive_df['label'] = 1 
negative_df =   pd.DataFrame(negative_train_tweets)
negative_df['label'] = 0 
df = pd.concat([positive_df ,negative_df], axis = 0 )
df = df.reset_index(drop=True)
df=df.rename(columns = {0:'text'})

##Define the functions for prepocessing text, creating sequences and padding

In [ ]:
def remove_URL(text): #remove urls
  url = re.compile(r'https?://\S+|www\.\S+')
  return url.sub(r"",text)

def remove_punct(text): #remove punctuations 
  translator = str.maketrans("","",string.punctuation)
  return text.translate(translator)

stop_words = set(stopwords.words('english')) 

def remove_stopwords(text): #remove stopwords 
  filtered_words = [word.lower() for word in text.split() if word.lower() not in stop_words]
  return " ".join(filtered_words)

def count_frequency(df,text): #count the frequency of each word 
  freq_dict = {}
  for row in df[text]:
    for word in row.split(): #tokenize
      if word not in freq_dict:
        freq_dict[word] = 1 
      else:
        freq_dict[word] += 1
  return freq_dict 

def find_unique_words(df,text): # find the unique words
  all_unique_words = []
  for row in df[text]:
    for word in row.split():
      if word not in all_unique_words:
        all_unique_words.append(word)
  all_unique_words = sorted(list(set(all_unique_words)))
  return all_unique_words

def index_assignment(all_unique_words): #assign index to each word
  index_dict = {}
  count = 0 
  for word in all_unique_words:
    count += 1 
    index_dict[word] = count 
  return index_dict

def text_to_sequences(df,text,index_dict): #convert text to sequence
  text_sequences = []
  for row in df[text]:
    empty = []
    for word in row.split():
      empty.append(index_dict.get(word))
    text_sequences.append(empty)
  return text_sequences

 
def padded_sequences(text_sequences:list): #pad sequences
  padded_sequences = []
  max_len = 30 #choose an arbritrary number 
  for text_sequence in text_sequences:
    length = len(text_sequence)
    i = 0 
    while  length + i < max_len:
      text_sequence.insert(i,0)
      i += 1
    padded_sequences.append(text_sequence)
  return padded_sequences 

In [ ]:
#Apply the functions to the thraining data 
df['text']  = df['text'].apply(lambda x : remove_URL(x))
df['text']  = df['text'].apply(lambda x : remove_punct(x))
df['text']  = df['text'].apply(lambda x : remove_stopwords(x))
frequencey_dict = count_frequency(df,'text')
unique_words = find_unique_words(df,'text')
index_dictionary = index_assignment(unique_words)
train_sequences = text_to_sequences(df,'text',index_dictionary)
train_padded = padded_sequences(train_sequences)

##Classify text using Logistic Regression 

In [ ]:
#Initialize the wights vector 
weights = []
for _ in range(30):
  weights.append(1/30)

#Training the model 
predictions = []
bias = 0.5
lr = 0.0001
m = 1 / len(train_padded[0])

for i,element in enumerate(train_padded):
  derivatives = []
  prediction = 0 
  actual = df['label'][i]
  for j,pad in enumerate(element):
    prediction += pad * weights[j]
  prediction += bias 
  sigmoid = 1 / (1 + np.exp(-prediction)) 
  predictions.append(round(sigmoid))

  for i in range(len(weights)):
    dw = (1/m) * (element[i] * (sigmoid - actual)) * lr 
    derivatives.append(dw)

  for i,derivative in enumerate(derivatives):
    weights[i] -= derivative 
  
  db = (1/m) * (sigmoid - actual) * lr

  bias -= db

df['predictions'] = predictions

print('Accuracy is: ' + str(df['label'].sum() / df['predictions'].sum()))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


Accuracy is: 0.9982530571499876
